# Notebook for DE Analysis of single-cell data

## Load Packages

In [1]:
suppressMessages(library(Seurat))
suppressPackageStartupMessages(library(DESeq2))
suppressMessages(library(dplyr))
suppressPackageStartupMessages(library(stringr))
suppressMessages(library(data.table))
suppressMessages(library(Matrix))

## Define Inputs

### Assumptions:
- The cell type column in your seurat object is called 'cell_type'
- The sample/donor ID in your seurat object is called 'library'

In [2]:
# Inputs: Seurat object and metadata
seurat_obj_path <- '/nfs/lab/sara/projects/PanKbase/singlecell/input/2023_01_27_hpap_res0p3.rds'
metadata_path <- '/nfs/lab/sara/projects/PanKbase/singlecell/input/Donor_Summary_127.csv'

# Load the Seurat object and metadata
data <- readRDS(seurat_obj_path)
meta <- read.csv(metadata_path)

#### Format Inputs - Optional

In [3]:
# TO USER - Ensure that the metadata has only the donors present in the seurat object - no more no less. Modify or comment out this chunk according to your needs
meta <- read.csv(metadata_path)
meta <- meta[meta$donor_ID %in% unique(data@meta.data$library), ]

dib_stat  <- data@meta.data[,c('library', 'Diabetes_Status_w_AAB', 'chemistry', 'tissue_source')]
dib_stat <- unique(dib_stat)
rownames(dib_stat) <- NULL

meta <- merge(meta, dib_stat, by.x = 'donor_ID', by.y='library')

meta <- rename(meta, diabetes_status = Diabetes_Status_w_AAB)
meta <- rename(meta, sex = gender)

rownames(meta) <- meta$donor_ID

In [4]:
head(meta)

,donor_ID,rrid,recovery_opo,donor_ID_second,unos_ID,sex,bmi,age_years,race,hba1c,⋯,beta_cell_count,delta_cell_count,epsilon_cell_count,pp_cell_count,total_events,weight,height,diabetes_status,chemistry,tissue_source
,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<dbl>,<int>,<chr>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>
HPAP-019,HPAP-019,RRID:SAMN19776450,,17D0406,NA,Male,29.80,22,Caucasian,5.2,⋯,38637,3327,180,2114,55007,NA,NA,AAB+,10Xv2,nPod
HPAP-020,HPAP-020,RRID:SAMN19776451,,18J0307,NA,Male,13.20,14,Caucasian,NA,⋯,249,14485,939,6706,202889,NA,NA,T1D,10Xv2,nPod
HPAP-021,HPAP-021,RRID:SAMN19776452,,18M1122,NA,Female,21.40,13,Caucasian,NA,⋯,76,3108,306,1897,8954,NA,NA,T1D,10Xv2,nPod
HPAP-022,HPAP-022,RRID:SAMN19776453,,18M2405,NA,Female,34.70,39,Caucasian,4.7,⋯,2788,34375,437,2070,315113,NA,NA,ND,10Xv2,UPenn
HPAP-023,HPAP-023,RRID:SAMN19776454,,18A0312,NA,Female,21.35,17,Caucasian,8.9,⋯,607,3659,448,25351,31682,NA,NA,T1D,10Xv2,nPod
HPAP-024,HPAP-024,RRID:SAMN19776455,,18A0813,NA,Male,24.30,18,Caucasian,5.5,⋯,29407,2183,307,7435,42501,NA,NA,AAB+,10Xv2,nPod


In [5]:
# Set cell identities
Idents(data) <- data@meta.data$cell_type

# Define the samples based on metadata
samples <- unique(meta$donor_ID)
unique_cell_types <- unique(data$cell_type)

# Default assay should be 'RNA'
DefaultAssay(data) <- 'RNA'
gex.counts <- GetAssayData(data, layer = 'counts')

# Create the output directory if it doesn't exist
output_dir <- '/nfs/lab/sara/projects/PanKbase/singlecell/analytical_library/output'

if (!dir.exists(output_dir)) {
  dir.create(output_dir, recursive = TRUE)
}

In [6]:
print(paste("Unique donors:", length(samples)))
print(paste("Unique cell types:", length(unique_cell_types)))

[1] "Unique donors: 65"
[1] "Unique cell types: 14"


## Make Pseudobulk Matrices for each cell type

In [7]:
# Output directory for cell counts
outdir_cell_counts <- file.path(output_dir, "cell_counts")
dir.create(outdir_cell_counts, showWarnings = FALSE)

# Loop over each unique cell type
for (cell_type in unique_cell_types) {
  # Get barcodes for the current cell type
  bcs <- names(Idents(data)[Idents(data) == cell_type])
  
  # Subset the gene expression counts matrix for the cell type
  counts <- gex.counts[, colnames(gex.counts) %in% bcs]
  
  # Initialize a matrix to store pseudobulk counts for each sample
  pseudobulk_counts <- matrix(0, nrow = nrow(gex.counts), ncol = length(samples))
  row.names(pseudobulk_counts) <- row.names(gex.counts)
  
  # Loop over each sample
  for (i in seq_along(samples)) {
    sample <- samples[i]
    
    # Get barcodes for the current sample
    sample_bcs <- row.names(data@meta.data[data@meta.data$library == sample,])
    
    # Subset counts for the current sample's barcodes
    counts_cut <- counts[, colnames(counts) %in% sample_bcs]
    
    # Sum the counts across the barcodes for the current sample
    if (is.null(ncol(counts_cut)) || ncol(counts_cut) == 0) {
      sum_counts <- rep(0, nrow(counts))
    } else {
      sum_counts <- rowSums(counts_cut)
    }
      
    # Add the summed counts to the pseudobulk matrix
    pseudobulk_counts[, i] <- sum_counts
  }
  
  # Convert the matrix to a dataframe and assign sample names as column names
  pseudobulk_df <- as.data.frame(pseudobulk_counts)
  colnames(pseudobulk_df) <- samples
  
  # Save the pseudobulk counts
  output_file <- file.path(outdir_cell_counts, paste0(cell_type, "_sample_gex_total_counts.txt"))
  write.table(pseudobulk_df, output_file, sep = '\t', quote = FALSE, col.names = TRUE, row.names = TRUE)
}

## Define helper functions for DESeq

In [8]:
# Function to read the paths of the count matrices from the dir
readCounts <- function(counts_dir) {
  # Get all files matching the pattern
  files <- list.files(counts_dir, pattern = "_sample_gex_total_counts.txt$", full.names = TRUE)
  
  # Initialize an empty dict to store the cell type and the respective counts matrices' file path
  counts_dict <- list()
  
  # Loop over each file and extract the cell type from the filename
  for (file_path in files) {
    cell_type <- gsub("_sample_gex_total_counts.txt", "", basename(file_path))
    
    # Read the count matrix and store it in dict
    counts_dict[[cell_type]] <- read.csv(paste0(file_path), header = TRUE, row.names=1, sep='\t', check.names = FALSE)
  }
  
  return(counts_dict)
}

In [9]:
# Scale covariates function
scaleCovariates <- function(coldata_subset, covariates) {
  if (!is.null(covariates)) {
    coldata_subset[covariates] <- lapply(coldata_subset[covariates], function(x) {
      if (is.numeric(x)) {
        return(scale(x))  # Scale numeric covariates
      }
      return(x)  # Return unchanged for non-numeric covariates
    })
  }
  return(coldata_subset)
}

In [10]:
# Function to filter samples based on the the min number of cells required per sample for a cell type
filterSamples  <- function(counts_matrix, coldata, min_cells, cell_type, condition_col, condition_levels) {
  # Get cell counts for each sample from the meta data
  cell_counts <- table(data@meta.data$library, data@meta.data$cell_type)[, cell_type]
  
  # Filter out samples that have fewer than min_cells for the given cell type
  valid_samples <- names(cell_counts[cell_counts >= min_cells])

  # Subset counts matrix and coldata based on valid samples
  counts_matrix <- counts_matrix[, colnames(counts_matrix) %in% valid_samples]
  coldata <- coldata[rownames(coldata) %in% valid_samples, ]

  # Print the number of valid samples for each condition level
  for (level in condition_levels) {
    samples_in_level <- rownames(coldata[coldata[[condition_col]] == level, ])
    num_samples <- length(samples_in_level)
    message(paste("Number of valid samples in condition level", level, "for", cell_type, "cells:", num_samples))
  }
    
  return(list(counts_matrix = counts_matrix, coldata = coldata))
}

## Perform DESeq

In [11]:
# Define the function to perform DESeq2 analysis
runDESeq2 <- function(counts_matrix, min_cells, coldata, condition_col, condition_levels, covariates, outdir_deseq, cell_type) {
  # Ensure the condition column exists in the coldata
  if (!(condition_col %in% colnames(coldata))) {
    stop(paste("Condition column", condition_col, "not found in coldata."))
  }

  # Passing arguments to the filterSamples() function 
  filtered_data <- filterSamples(counts_matrix, coldata, min_cells, cell_type, condition_col, condition_levels)
  counts_matrix <- filtered_data$counts_matrix
  coldata <- filtered_data$coldata
  
  # Ensure there are min of 2 samples in both condition levels
  # DESeq2 requires at least two samples per condition level to estimate dispersion 
  sample_counts <- table(coldata[[condition_col]])
  if (any(sample_counts[condition_levels] < 2)) { 
    message(paste("Skipping DESeq2 for", cell_type, "- minimum 2 samples required in one or both conditions after sample filtering."))
    return(NULL)
  }

  # Subset the coldata for the specified condition levels (e.g., T1D vs ND)
  coldata_subset <- coldata[coldata[[condition_col]] %in% condition_levels, ]

  # Match the subset of coldata with the counts matrix
  counts_subset <- counts_matrix[, rownames(coldata_subset)]
    
  # Calculate half the sample size for each condition_levels - used as the minimum number of samples to meet gene count thresholds
  n_levels <- sapply(condition_levels, function(level) {
    sum(coldata_subset[[condition_col]] == level)
  })
  n_levels_half <- floor(n_levels/2)
    
  # Filter genes based on the counts criteria - at least half the samples per condition have greater than 5 counts
  genes_to_keep <- rownames(counts_subset)[
    rowSums(counts_subset[, coldata_subset[[condition_col]] == condition_levels[1]] >= 5) >= n_levels_half[1] &
    rowSums(counts_subset[, coldata_subset[[condition_col]] == condition_levels[2]] >= 5) >= n_levels_half[2]
  ]

  # Subset counts matrix to keep only the genes that passed the filter
  counts_subset <- counts_subset[genes_to_keep, ]

  # Re-level the factor so the first condition is the reference level
  coldata_subset[[condition_col]] <- factor(coldata_subset[[condition_col]], levels = condition_levels)

  # Build the design formula with covariates
  if (!is.null(covariates) && all(covariates %in% colnames(coldata_subset))) {
    design_formula <- as.formula(paste("~", paste(c(covariates, condition_col), collapse = " + ")))
  } else {
    design_formula <- as.formula(paste("~", condition_col))
  }
    
  # Create DESeq2 dataset
  dds <- DESeqDataSetFromMatrix(countData = counts_subset, colData = coldata_subset, design = design_formula)

  # Run DESeq2
  dds <- DESeq(dds)

  # Extract results for the specified condition comparison
  res <- results(dds, contrast = c(condition_col, condition_levels[1], condition_levels[2]))
  res <- res[order(res$pvalue), ]

  # Save results
  condition_comparison <- paste(condition_levels[1], "vs", condition_levels[2], sep = "_")
  output_filename <- paste0(outdir_deseq, "/", cell_type, "_", condition_comparison, "_deseq.tsv")

  write.table(res, output_filename, sep = "\t", quote = FALSE, row.names = TRUE, col.names = TRUE)

  message(paste("Complete! DESeq analysis for", condition_levels[1], "vs", condition_levels[2], "in", cell_type, "cells."))

  return(dds)
}

In [12]:
run_DESeq2_for_all_cell_types <- function(cell_types_list = NULL, min_cells = 10, counts_dir, coldata, condition_col, condition_levels, covariates = NULL, output_dir) {
  # Ensure the base output directory exists or create it
  outdir_deseq <- file.path(output_dir, "deseq")
  if (!dir.exists(outdir_deseq)) {
    dir.create(outdir_deseq, recursive = TRUE)
  }
  
  # If cell_types_list is not provided, read all count matrices from the directory
  if (is.null(cell_types_list)) {
    counts_dict <- readCounts(counts_dir)
    cell_types_list <- names(counts_dict)  # Use names of the counts_dict as cell types
  } else {
    counts_dict <- readCounts(counts_dir)[cell_types_list]  # Only read specified cell types
  }
  
  # Loop over each cell type and run DESeq2
  for (cell_type in cell_types_list) {
    # Retrieve the counts matrix for the current cell type
    counts_matrix <- counts_dict[[cell_type]]

    # Skip if the counts data for the cell type is missing
    if (is.null(counts_matrix)) {
      message(paste("Skipping DESeq2 for", cell_type, "due to missing counts data."))
      next
    }
    
    # Run DESeq2 for the current cell type and save results
    runDESeq2(counts_matrix = counts_matrix, min_cells = min_cells, coldata = coldata, condition_col = condition_col, 
               condition_levels = condition_levels, covariates = covariates, 
               outdir_deseq = outdir_deseq, cell_type = cell_type)
  }
  
  message("DESeq2 analysis completed for specified cell types.")
}

## Custom DESeq Parameters

#### Required Arguments
- **coldata**: metadata file that includes sample information, conditions and covariates for DESeq analysis
- **condition_col**: The column name in coldata that represents the experimental condition to test (e.g., "diabetes_status")
- **condition_levels**: A vector specifying the two condition levels to compare in DESeq (e.g., c("T1D", "ND"))
- **counts_dir**: The directory where the pseudobulk matrices for each cell type will be saved
- **output_dir**: The directory where DESeq output will be saved

#### Optional Arguments
- **cell_types_list**: (default = NULL) list of specific cell types of interest to run DESeq on. If not provided, all cell types in counts_dir are used.
- **covariates**: (default = NULL) A list of additional covariates from coldata to include in the DESeq model (e.g., age, sex)
- **min_cells**: (default = 10) sets threshold for the minimum number of cells required per sample for a cell type. Samples with fewer cells are removed from analysis

In [13]:
### Required
coldata <- meta  # Sample metadata with condition and covariates
condition_col <- "diabetes_status"
condition_levels <- c("T1D", "ND")
counts_dir <- "/nfs/lab/sara/projects/PanKbase/singlecell/analytical_library/output/cell_counts"
output_dir <- "/nfs/lab/sara/projects/PanKbase/singlecell/analytical_library/output" 

### Optional
cell_types_list <- c("Beta")
covariates <- c("age_years", "sex", "bmi", "chemistry", "tissue_source")
min_cells <- 5

# # To run DESeq2 for specific cell types
# run_DESeq2_for_all_cell_types(cell_types_list = cell_types_list, 
#                               coldata = coldata,
#                               condition_col = condition_col, 
#                               condition_levels = condition_levels, 
#                               counts_dir = counts_dir, 
#                               output_dir = output_dir,                              
#                               min_cells = min_cells,                               
#                               covariates = covariates)


# To run DESeq2 for all cell types - entire cell_counts dir
run_DESeq2_for_all_cell_types(coldata = coldata,
                              condition_col = condition_col, 
                              condition_levels = condition_levels, 
                              counts_dir = counts_dir, 
                              output_dir = output_dir,                              
                              min_cells = min_cells,                               
                              covariates = covariates)

Number of valid samples in condition level T1D for Acinar cells: 10

Number of valid samples in condition level ND for Acinar cells: 27

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  the design formula contains one or more numeric variables with integer values,
  specifying a model with increasing fold change for higher values.
  did you mean for this to be a factor? if so, first convert
  this variable to a factor using the factor() function

  the design formula contains one or more numeric variables that have mean or
  standard deviation larger than 5 (an arbitrary threshold to trigger this message).
  Including numeric variables with large mean can induce collinearity with the intercept.
  Users should center and scale numeric variables in the design to improve GLM convergence.

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

